In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import tensorflow as tf

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import sys

In [5]:
from tensorflow.keras.applications import EfficientNetB7

model = EfficientNetB7(weights = 'imagenet', include_top = False, input_shape = (64, 64, 3))

In [7]:
from tensorflow.keras import layers, models

# Add custom classification layers on top of EfficientNet
x = layers.GlobalMaxPooling2D()(model.output)  # Add global pooling layer
x = layers.Dense(128, activation='relu')(x)        # Add fully connected layer
x = layers.Dropout(0.5)(x)                         # Dropout for regularization
output = layers.Dense(1, activation='sigmoid')(x)  # Binary classification (sigmoid)

# Create the final model
model = models.Model(inputs=model.input, outputs=output)


In [9]:
model.compile(
    loss='binary_crossentropy',  # Use 'categorical_crossentropy' for multi-class classification
    optimizer='adam',            # You can experiment with different optimizers like SGD or RMSprop
    metrics=['accuracy']
)


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2   # Use 20% of the training data for validation
)

# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    '/Users/yeshnavya/Documents/archive_resnet/train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='training'         # Use the remaining 80% for training
)

validation_generator = train_datagen.flow_from_directory(
    '/Users/yeshnavya/Documents/archive_resnet/train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='validation'       # Use the 20% split for validation
)


Found 80000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [13]:
from tensorflow.keras.callbacks import EarlyStopping

# Define EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor (can be 'val_accuracy' or 'accuracy' if no validation set)
    patience=3,          # Number of epochs to wait before stopping after no improvement
    restore_best_weights=True  # Restore the weights of the best model after stopping
)

history = model.fit(
    train_generator,
    epochs=10,                    # Number of training epochs
    validation_data=validation_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator)
)


Epoch 1/10


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2500/2500 ━━━━━━━━━━━━━━━━━━━━ 7966s 3s/step - accuracy: 0.8937 - loss: 0.2772 - val_accuracy: 0.9467 - val_loss: 0.1471
Epoch 2/10


2024-10-04 02:08:20.242547: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/anaconda3/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
2024-10-04 02:08:20.360284: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2500/2500 ━━━━━━━━━━━━━━━━━━━━ 0s 158us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 4754s 2s/step - accuracy: 0.9352 - loss: 0.1699 - val_accuracy: 0.9459 - val_loss: 0.1375
Epoch 4/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 0s 66us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10


2024-10-04 03:27:34.391820: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2500/2500 ━━━━━━━━━━━━━━━━━━━━ 4510s 2s/step - accuracy: 0.9574 - loss: 0.1201 - val_accuracy: 0.6714 - val_loss: 3.4989
Epoch 6/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 0s 51us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 5408s 2s/step - accuracy: 0.9590 - loss: 0.1115 - val_accuracy: 0.9442 - val_loss: 1.7099
Epoch 8/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 0s 61us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10


2024-10-04 06:12:53.090709: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2500/2500 ━━━━━━━━━━━━━━━━━━━━ 12003s 5s/step - accuracy: 0.9676 - loss: 0.0899 - val_accuracy: 0.9548 - val_loss: 0.2312
Epoch 10/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 0s 57us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00


In [18]:
model.save('/Users/yeshnavya/Documents/venv/EfficientNetB7_Fake_Real_final.h5')